<a href="https://colab.research.google.com/github/kankshamasrani/Machine-Learning-A2/blob/master/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
NUM_EPOCH = 10
use_cuda = True

class ResNet50_CIFAR(nn.Module):
    def __init__(self):
        super(ResNet50_CIFAR, self).__init__()
        # Initialize ResNet 50 with ImageNet weights
        ResNet50 = models.resnet50(pretrained=True)
        modules = list(ResNet50.children())[:-1]
        backbone = nn.Sequential(*modules)
        # Create new layers
        self.backbone = nn.Sequential(*modules)
        self.fc1 = nn.Linear(2048, 32)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, img):
        # Get the flattened vector from the backbone of resnet50
        out = self.backbone(img)
        # processing the vector with the added new layers
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.dropout(out)
        return self.fc2(out)

def train():
    ## Define the training dataloader
    transform = transforms.Compose([transforms.Resize(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5),
                                                         (0.5, 0.5, 0.5))])
    trainset = datasets.CIFAR10('./data', download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

    ## Create model, objective function and optimizer
    model = ResNet50_CIFAR()
    model = model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(list(model.fc1.parameters()) + list(model.fc2.parameters()),
                           lr=0.001, momentum=0.9)

    ## Do the training
    for epoch in range(NUM_EPOCH):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 20 == 19:    # print every 20 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
                running_loss = 0.0
    torch.save(model.state_dict(),'ResNetTrained')
    print('Finished Training')




if __name__ == '__main__':
    train()


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:02<00:00, 35906089.16it/s]


[1,    20] loss: 2.356
[1,    40] loss: 2.459
[1,    60] loss: 2.307
[1,    80] loss: 2.321
[1,   100] loss: 2.258
[1,   120] loss: 2.243
[1,   140] loss: 2.116
[1,   160] loss: 2.091
[1,   180] loss: 2.065
[1,   200] loss: 2.164
[1,   220] loss: 1.950
[1,   240] loss: 2.168
[1,   260] loss: 1.882
[1,   280] loss: 1.892
[1,   300] loss: 1.839
[1,   320] loss: 1.918
[1,   340] loss: 1.833
[1,   360] loss: 1.890
[1,   380] loss: 1.854
[1,   400] loss: 1.798
[1,   420] loss: 1.686
[1,   440] loss: 1.792
[1,   460] loss: 1.716
[1,   480] loss: 1.725
[1,   500] loss: 1.662
[1,   520] loss: 1.776
[1,   540] loss: 1.675
[1,   560] loss: 1.741
[1,   580] loss: 1.787
[1,   600] loss: 1.845
[1,   620] loss: 1.707
[1,   640] loss: 1.785
[1,   660] loss: 1.791
[1,   680] loss: 1.950
[1,   700] loss: 1.602
[1,   720] loss: 1.590
[1,   740] loss: 1.647
[1,   760] loss: 1.683
[1,   780] loss: 1.692
[1,   800] loss: 1.617
[1,   820] loss: 1.432
[1,   840] loss: 1.531
[1,   860] loss: 1.636
[1,   880] 

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch